In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
p = np.linspace(0, 20, 100)
plt.plot(p, np.sin(15*p))

plt.show()

In [1]:
import pandas as pd
import pandas_datareader as pdr
v = pdr.get_data_fred('GS10')


In [ ]:
v